In [4]:
import random
import time
import uuid



In [17]:
def randomInt():
    return random.randint(-2147483648, 2147483647)

In [35]:
deviceNameLength = 32
serviceNameLength = 8

devicePort = 8080

In [5]:
def generateName(length=5):
    return str(uuid.uuid4().hex)[:length]



In [33]:
class ArduinoDevice:
    def __init__(self, name, controllers,serialName=None, rx=8, tx=9):
        self.rx = rx
        self.tx = tx
        self.name = name
        self.serialname = generateName(deviceNameLength) if (serialName == None) else serialName
        self.type = "arduino"
        self.port = devicePort
        self.controllers = controllers
        
        

In [26]:
intypes = ["analog", "temperature11", "humidity11", "temperature22", "humidity22", "alert", "pressure"]

In [42]:
ids = {'analog': 1000,
 'onoff': 1001,
 'temperature11': 1002,
 'humidity11': 1003,
 'alert': 1004,
 'temperature22': 1006,
 'humidity22': 1007,
 'pressure': 1008}

serviceIds = {
    'analog': "ANALOG_ID",
 'onoff':"ONOFF_ID",
 'temperature11': "TEMPERATURE_DHT11_ID",
 'humidity11': "HUMIDITY_DHT11_ID",
 'alert': "DIGITAL_ALERT_ID",
 'temperature22': "TEMPERATURE_DHT22_ID",
 'humidity22': "HUMIDITY_DHT22_ID",
 'pressure': "PRESSURE_ID"
}


In [62]:
class ArduinoController:
    def __init__(self, _type, pin):
        self._type =ids[_type]
        self.rawType = _type
        self.serviceId = serviceIds[_type]
        self.defineService = serviceIds[_type][:-3]
        self.pin = pin
        self.serial = randomInt()
        
        

In [63]:
device = ArduinoDevice(name="1", controllers=[
    ArduinoController("analog", "A3"),
    ArduinoController("alert", "10"),
    ArduinoController("alert", "12"),
    ArduinoController("temperature22", "7"),
    ArduinoController("humidity22", "7"),
    ArduinoController("onoff", "4"),
    ArduinoController("onoff", "3"),
    ArduinoController("onoff", "2")
])


In [64]:
paste = """
#include "user_properties.h"
#include "services.h"

#define RASPBERRY_PORT 8080
#define RASPBERRY_IP "192.168.0.102"
#define ARDUINO_PORT 80

// used services
"""

for i in set(map(lambda x: x.defineService, device.controllers)):
    paste += "#define " + i + "\n"
    



In [54]:
def mode(controller):
    return "INPUT" if controller.rawType in intypes else "OUTPUT"

In [65]:
paste = """
#include "user_properties.h"
#include "services.h"
#include "service.h"

#define RASPBERRY_PORT 8080
#define RASPBERRY_IP "192.168.0.102"
#define ARDUINO_PORT 80

// used services
"""

for i in set(map(lambda x: x.defineService, device.controllers)):
    paste += "#define " + i + "\n"


    
paste += "\n// pins for communication with esp8266\n"
paste += "#define RX " + str(device.rx) + " // attach to TX of esp\n"
paste += "#define TX " + str(device.tx) + " // attach to RX of esp\n"
paste += "Service services[10];"
paste += "void setupConfiguration() {\n"

for controller in device.controllers:
    paste += "\tpinMode(" + str(controller.pin) + ", " + mode(controller) + ");\n"
    paste += "\tservices[%i].pin = %s;\n" % (device.controllers.index(controller), controller.pin)
    paste += "\tservices[%i].type = %s;\n" % (device.controllers.index(controller), controller._type)
    paste += "\tservices[%i].serial = %s;\n" % (device.controllers.index(controller), controller.serial)
    paste += "\n"
paste += "}\n"

print(paste)


#include "user_properties.h"
#include "services.h"

#define RASPBERRY_PORT 8080
#define RASPBERRY_IP "192.168.0.102"
#define ARDUINO_PORT 80

// used services
#define DIGITAL_ALERT
#define ONOFF
#define TEMPERATURE_DHT22
#define HUMIDITY_DHT22
#define ANALOG

// pins for communication with esp8266
#define RX 8 // attach to TX of esp
#define TX 9 // attach to RX of esp
Service services[10];void setupConfiguration() {
	pinMode(A3, INPUT);
	services[0].pin = A3;
	services[0].type = 1000;
	services[0].serial = 1717276558;

	pinMode(10, INPUT);
	services[1].pin = 10;
	services[1].type = 1004;
	services[1].serial = -540306999;

	pinMode(12, INPUT);
	services[2].pin = 12;
	services[2].type = 1004;
	services[2].serial = 940716679;

	pinMode(7, INPUT);
	services[3].pin = 7;
	services[3].type = 1006;
	services[3].serial = -634933851;

	pinMode(7, INPUT);
	services[4].pin = 7;
	services[4].type = 1007;
	services[4].serial = -539064424;

	pinMode(4, OUTPUT);
	services[5].pin = 4;
	services[5].typ